In [14]:
import os
 
import requests
import time
from datetime import datetime
from requests.adapters import HTTPAdapter
import pandas as pd
 
pd.set_option('display.max_rows', 1000)
pd.set_option('expand_frame_repr', False)  # 当列太多时不换行
# 设置命令行输出时的列对齐功能
pd.set_option('display.unicode.ambiguous_as_wide', True)
pd.set_option('display.unicode.east_asian_width', True)


In [15]:
def requestfornew(url,max_try_num=10,sleep_time=5):
    headers={
        'Referer': 'http://finance.sina.com.cn',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.62'
    }
    for i in range(max_try_num):
        response=requests.get(url,headers=headers,timeout=30)
        if response.status_code==200:
            return response
        else:
            print('链接失败',response)
            time.sleep(sleep_time)


In [16]:
def getdate():
    url='https://hq.sinajs.cn/list=sh000001'
    response=requestfornew(url).text
    data_date=str(response.split(',')[-4])
    return data_date

In [17]:
def getstockcodeforeveryday():
    df = pd.DataFrame()
    page = 1
    while True:
        # 1~100页，不用担心每天新增
        url = 'http://vip.stock.finance.sina.com.cn/quotes_service/api/json_v2.php/Market_Center.getHQNodeData?page=' \
              + str(page) + '&num=80&sort=changepercent&asc=0&node=hs_a&symbol=&_s_r_a=page'
        # print(url)
        content = requestfornew(url).json()
        if not content:
            # if content =[]: 这个写法也可以
            print("股票信息，获取完毕。")
            break
        print("正在读取页面" + str(page))
        time.sleep(3)
        df = df.append(pd.DataFrame(content, dtype='float'), ignore_index=True)
        page += 1
 
    rename_dict = {'symbol': '股票代码', 'code': '交易日期', 'name': '股票名称', 'open': '开盘价',
                   'settlement': '前收盘价', 'trade': '收盘价', 'high': '最高价', 'low': '最低价',
                   'buy': '买一', 'sell': '卖一', 'volume': '成交量', 'amount': '成交额',
                   'changepercent': '涨跌幅', 'pricechange': '涨跌额',
                   'mktcap': '总市值', 'nmc': '流通市值', 'ticktime': '数据更新时间', 'per': 'per', 'pb': '市净率',
                   'turnoverratio': '换手率'}
    df.rename(columns=rename_dict, inplace=True)
    tradedate = getdate()
    df['交易日期'] = tradedate
    df['总市值'] = df['总市值'] * 10000
    df['流通市值'] = df['流通市值'] * 10000
    df = df[['股票代码', '股票名称', '交易日期', '开盘价', '最高价', '最低价', '收盘价', '前收盘价', '成交量', '成交额', '流通市值', '总市值']]
    # 把转化成float的code替换成交易日期
    return df

In [18]:
df = getstockcodeforeveryday()
df = df[df['开盘价'] - 0 > 0.00001]
df.reset_index(drop=True, inplace=True) # 删除旧的索引，从头开始
print(df)

正在读取页面1
正在读取页面2
正在读取页面3
正在读取页面4
正在读取页面5
正在读取页面6
正在读取页面7
正在读取页面8
正在读取页面9
正在读取页面10
正在读取页面11
正在读取页面12
正在读取页面13
正在读取页面14
正在读取页面15
正在读取页面16
正在读取页面17
正在读取页面18
正在读取页面19
正在读取页面20
正在读取页面21
正在读取页面22
正在读取页面23
正在读取页面24
正在读取页面25
正在读取页面26
正在读取页面27
正在读取页面28
正在读取页面29
正在读取页面30
正在读取页面31
正在读取页面32
正在读取页面33
正在读取页面34
正在读取页面35
正在读取页面36
正在读取页面37
正在读取页面38
正在读取页面39
正在读取页面40
正在读取页面41
正在读取页面42
正在读取页面43
正在读取页面44
正在读取页面45
正在读取页面46
正在读取页面47
正在读取页面48
正在读取页面49
正在读取页面50
正在读取页面51
正在读取页面52
正在读取页面53
正在读取页面54
正在读取页面55
正在读取页面56
正在读取页面57
正在读取页面58
正在读取页面59
正在读取页面60
正在读取页面61
正在读取页面62
正在读取页面63
正在读取页面64
正在读取页面65
正在读取页面66
股票信息，获取完毕。
      股票代码  股票名称    交易日期  开盘价  最高价  最低价  收盘价  前收盘价      成交量       成交额      流通市值        总市值
0     sz301255  通力科技  2023-06-15   26.52   31.18   26.52   31.18     25.98   7559724.0  227456797.0  8.480960e+08  3.392384e+09
1     sz300946    恒而达  2023-06-15   32.17   38.59   31.98   38.59     32.16   7919065.0  283420602.0  1.317694e+09  4.631032e+09
2     sz300660  江苏雷利  2023-06-15   27.20   32.71   27.07

In [19]:
df

,股票代码,股票名称,交易日期,开盘价,最高价,最低价,收盘价,前收盘价,成交量,成交额,流通市值,总市值
0,sz301255,通力科技,2023-06-15,26.52,31.18,26.52,31.18,25.98,7559724.0,227456797.0,8.480960e+08,3.392384e+09
1,sz300946,恒而达,2023-06-15,32.17,38.59,31.98,38.59,32.16,7919065.0,283420602.0,1.317694e+09,4.631032e+09
2,sz300660,江苏雷利,2023-06-15,27.20,32.71,27.07,32.71,27.26,22240311.0,674147799.0,1.028467e+10,1.028908e+10
3,sz301102,兆讯传媒,2023-06-15,36.00,43.22,35.88,43.22,36.02,5008848.0,201793585.0,2.124461e+09,8.644000e+09
4,sz300615,欣天科技,2023-06-15,16.49,19.82,16.33,19.82,16.52,24586689.0,468248519.0,2.550150e+09,3.764754e+09
...,...,...,...,...,...,...,...,...,...,...,...,...
5171,sh603159,上海亚虹,2023-06-15,18.03,18.03,15.75,15.76,17.50,12799776.0,209171211.0,2.206400e+09,2.206400e+09
5172,sh600822,上海物贸,2023-06-15,15.62,15.89,15.44,15.44,17.15,40603684.0,628909177.0,6.116524e+09,7.657822e+09
5173,sh688435,英方软件,2023-06-15,96.00,96.00,80.03,82.86,94.99,2395760.0,207911473.0,1.459787e+09,6.918810e+09
5174,sz002417,深南退,2023-06-15,0.35,0.40,0.34,0.39,1.24,47601142.0,17129785.0,9.198397e+07,1.054064e+08


In [20]:
for i in df.index:
    t = df.iloc[i:i + 1, :]
    stock_code = t.iloc[0]['股票代码']
 
    # 构建存储文件路径
    path = './data/' \
           + stock_code + '.csv'
    # 文件存在，不是新股
    if os.path.exists(path):
        t.to_csv(path, header=None, index=False, mode='a', encoding='gbk')
    # 文件不存在，说明是新股
    else:
        # 先将头文件输出
        pd.DataFrame(columns=['股票']).to_csv(path, index=False, encoding='gbk')
        t.to_csv(path, index=False, mode='a', encoding='gbk')
    print(stock_code)

sz301255
sz300946
sz300660
sz301102
sz300615
bj836221
sz300273
sz300503
sz300565
sh688408
sh688003
sh688160
bj873593
sz300116
sh688337
sz300403
sz300880
sz301266
sz301168
sh688320
sz000606
sh688515
sz002535
sz002177
sz002527
sz002877
sz002725
sz002068
sz000936
sz002553
sz002962
sh603803
sh603861
sh603015
sz002380
sh603662
sh605589
sz002281
sh603322
sh603728
sz002902
sz002896
sh603638
sz002184
sz002585
sz002747
sh603269
sh600322
sz002146
sh688032
sz300336
sh688503
sh688017
sz002865
sh688348
sh688556
sh600845
sh601949
bj839167
sh688165
sh601127
sz301320
sz002959
sh688195
sz300450
sz301368
sz002398
sz300488
sh688027
sz300698
sh688662
sh601689
sz002046
bj871478
sz300913
sh688218
sz300825
sh688472
sz300024
sz002164
sz300402
sz300125
sz002019
sz300217
sz300686
sz300850
sh603613
sz300549
sh688048
sz002459
sz300853
sh605277
sh603011
sz002821
sh603063
sz301023
sz300882
sz002864
sz002031
sh688223
sh600487
sh601865
sz300757
sz300081
sz002050
sz002850
sh688326
sz300607
sh605168
sz300049
sh603000
s